In [7]:
import numpy as np
import pandas as pd
import pickle
import re, nltk, spacy, gensim
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from gensim.utils import simple_preprocess
from pprint import pprint
from sklearn.metrics.pairwise import euclidean_distances
from nltk.corpus import stopwords

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
df = pd.read_csv('../Data/mortgage.csv')
df.head()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/28/19,Mortgage,Conventional home mortgage,Struggling to pay mortgage,None,"In dispute of the loan # XXXX, for XXXX XXXX t...",Company believes it acted appropriately as aut...,"SELECT PORTFOLIO SERVICING, INC.",GA,300XX,None,Consent provided,Web,03/28/19,Closed with explanation,Yes,NaN,3193709
1,02/21/19,Mortgage,FHA mortgage,Struggling to pay mortgage,None,My complaint is with Carrington Mortgage Servi...,None,"CARRINGTON MORTGAGE SERVICES, LLC",GA,312XX,None,Consent provided,Web,02/21/19,Closed with explanation,Yes,NaN,3158652
2,06/11/19,Mortgage,Home equity loan or line of credit (HELOC),Trouble during payment process,None,As of XX/XX/2019 PHH Mortgage Services purchas...,None,PHH Mortgage Services Corporation,FL,347XX,None,Consent provided,Web,06/11/19,Closed with explanation,Yes,NaN,3270600
3,11/29/19,Mortgage,Conventional home mortgage,Trouble during payment process,None,We started our mortgage with Loan Depot in XX/...,Company believes complaint is the result of an...,"LD Holdings Group, LLC",NJ,None,None,Consent provided,Web,11/29/19,Closed with monetary relief,Yes,NaN,3453669
4,10/23/19,Mortgage,Conventional home mortgage,Struggling to pay mortgage,None,"XXXX XXXX XXXX XXXX XXXX XXXX XXXX XXXX, XXXX ...",None,"Community Loan Servicing, LLC (formerly known ...",CA,92021,None,Consent provided,Web,11/05/19,Closed with explanation,Yes,NaN,3415392


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32805 entries, 0 to 32804
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 32805 non-null  object 
 1   Product                       32805 non-null  object 
 2   Sub-product                   32805 non-null  object 
 3   Issue                         32805 non-null  object 
 4   Sub-issue                     32805 non-null  object 
 5   Consumer complaint narrative  32805 non-null  object 
 6   Company public response       32805 non-null  object 
 7   Company                       32805 non-null  object 
 8   State                         32805 non-null  object 
 9   ZIP code                      32805 non-null  object 
 10  Tags                          32805 non-null  object 
 11  Consumer consent provided?    32805 non-null  object 
 12  Submitted via                 32805 non-null  object 
 13  D

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df['Issue'].value_counts()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Trouble during payment process                                                      15884
Struggling to pay mortgage                                                           7092
Applying for a mortgage or refinancing an existing mortgage                          5085
Closing on a mortgage                                                                3615
Incorrect information on your report                                                  677
Problem with a credit reporting company's investigation into an existing problem      335
Improper use of your report                                                            75
Unable to get your credit report or credit score                                       19
Credit monitoring or identity theft protection services                                16
Problem with fraud alerts or security freezes                                           7
Name: Issue, dtype: int64

In [11]:
df = df.drop(['Date received', 'Sub-issue', 'Company public response', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID'], axis=1)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
data = df['Consumer complaint narrative'].tolist()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
#data = [sent.lower() for sent in data] #lowecase
data = [re.sub(r'\s+', ' ', sent) for sent in data]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
# Tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
len(data_words[0])

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


722

In [9]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'xxxx', 'xx', 'well', 'fargo', 'citibank', 'mortgage'])
for key in df['Company'].value_counts().keys():
    stop_words.extend([key])

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data_words = remove_stopwords(data_words)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN','VERB']) #select noun and verb

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10, 
                             token_pattern='[a-zA-Z0-9]{3,}',  
                             max_features=50000
                            )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
search_params = {'n_components': [8, 10, 12], 'learning_decay': [0.7]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.7], 'n_components': [8, 10, 12]})

In [15]:
# Best LDA model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 10}
Best Log Likelihood Score:  -4235363.706805143
Model Perplexity:  488.2539775368297


In [16]:
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return "color: {col}".format(col=color)
def make_bold(val):
    weight = 700 if val > .1 else 400
    return "font-weight: {weight}".format(weight=weight)

df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,dominant_topic
Doc0,0.170000,0.380000,0.080000,0.000000,0.010000,0.050000,0.310000,0.010000,0.000000,0.000000,1
Doc1,0.010000,0.260000,0.010000,0.010000,0.010000,0.700000,0.010000,0.010000,0.010000,0.010000,5
Doc2,0.050000,0.040000,0.140000,0.000000,0.000000,0.490000,0.000000,0.040000,0.030000,0.200000,5
Doc3,0.000000,0.000000,0.160000,0.000000,0.000000,0.530000,0.000000,0.230000,0.070000,0.000000,5
Doc4,0.060000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.680000,9
Doc5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.650000,0.250000,0.080000,7
Doc6,0.350000,0.460000,0.030000,0.050000,0.000000,0.000000,0.090000,0.000000,0.000000,0.020000,1
Doc7,0.000000,0.000000,0.320000,0.000000,0.000000,0.290000,0.000000,0.170000,0.000000,0.230000,2
Doc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.730000,0.000000,0.250000,0.000000,0.000000,5
Doc9,0.370000,0.070000,0.020000,0.000000,0.010000,0.000000,0.050000,0.000000,0.180000,0.300000,0


In [17]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic Num,Num Documents
0,5,7353
1,9,5653
2,2,4670
3,6,4137
4,0,3515
5,7,2788
6,8,2746
7,1,1751
8,4,180
9,3,12


In [19]:
def show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,loan,modification,mortgage,payment,foreclosure,home,year,servicing,make,servicer,ocwen,month,pmi,pay,help
Topic 1,mortgage,property,bank,attorney,file,document,note,law,court,state,debt,bankruptcy,ditech,deed,claim
Topic 2,loan,closing,rate,mortgage,refinance,credit,lender,email,process,document,appraisal,close,application,provide,officer
Topic 3,fund,rushmore,wire,quicken,payoff,bayview,citizen,funding,union,management,loan,rocket,veteran,hurricane,navy
Topic 4,well,fargo,home,exhibit,mortgage,year,case,complaint,interest,letter,loan,rate,preservation,wfhm,continue
Topic 5,payment,mortgage,pay,make,month,bank,account,amount,fee,time,loan,call,tell,company,get
Topic 6,payment,receive,account,mortgage,request,letter,send,credit,date,statement,loan,information,report,make,service
Topic 7,escrow,insurance,pay,taxis,tax,property,mortgage,account,company,amount,policy,year,increase,receive,refund
Topic 8,home,forbearance,sale,house,income,request,time,property,work,application,value,submit,offer,process,need
Topic 9,call,tell,get,say,send,receive,ask,time,speak,day,email,check,phone,need,company


In [20]:
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,abandon,abandonment,abatement,abide,ability,able,absence,absolve,absorb,abundance,...,yard,year,yell,yesterday,yield,york,yrs,ytd,zip,zone
Topic0,36.152858,0.100091,0.100018,93.136786,212.151914,21.442146,0.100056,0.100018,2.893977,0.100020,...,0.100554,5162.964187,0.100006,0.100024,23.398363,15.406197,82.984617,0.974086,0.602587,0.100007
Topic1,69.794551,0.100103,0.100004,0.100061,0.100047,0.100017,35.323269,10.424839,0.100075,22.724088,...,0.100025,487.396268,0.100010,0.100026,0.100047,263.454711,0.100036,0.100024,48.548263,0.100010
Topic2,10.303567,0.100008,0.100014,17.571776,144.979902,54.714992,14.298293,0.100096,13.972304,2.802888,...,0.100011,1691.974314,0.105779,64.940266,0.100151,0.100029,6.415056,0.100044,11.496533,0.100014
Topic3,0.100047,0.100010,0.100017,0.100178,0.100028,0.100048,7.286394,0.100011,0.100024,0.100116,...,0.100041,0.107827,0.100019,0.100027,0.100118,0.100028,0.100031,0.100005,0.100020,171.153899
Topic4,0.100017,0.100040,0.100000,0.100026,7.177530,0.100036,5.635254,0.100005,0.100109,0.100043,...,0.100031,637.962534,0.100011,0.100038,0.100123,0.994170,1.017315,0.100195,0.100015,0.100010


In [21]:
#Save best model
pickle.dump(model.best_estimator_, open('best-model-n10', 'wb'))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#Save grid search model
#pickle.dump(model, open('sklearn-grid-model-n3-7-10', 'wb'))

In [24]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      32.420952   8.115230       1        1  18.152311
5       0.648955  71.707100       2        1  15.990276
6      71.019684   9.971006       3        1  14.032141
2       3.785981  33.185135       4        1  12.780934
8     -42.666500  -9.222070       5        1  10.219062
0      -3.621689 -43.526516       6        1  10.127384
1      -3.632402  -4.171708       7        1   8.440168
7     -36.584400  35.804413       8        1   7.504417
4      44.391380  50.615383       9        1   1.527581
3      40.235809 -31.591665      10        1   1.225725, topic_info=           Term          Freq         Total Category  logprob  loglift
2918    payment  89927.000000  89927.000000  Default  30.0000  30.0000
2374       loan  80203.000000  80203.000000  Default  29.0000  29.0000
1456     escrow  21346.000000  21346.000000  Default  28.0000  28.0000
2142  insurance  18056.000000  18056.000000  Default  27.0000  27.0000
4477       well  10367.000000  10367.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1661    florida    193.592837    720.462905  Topic10  -5.3640   3.0875
1607       fema    177.922887    312.378303  Topic10  -5.4484   3.8388
4246   transfer    204.841284   9382.023007  Topic10  -5.3075   0.5773
3730    service    183.257075  14169.241247  Topic10  -5.4189   0.0537
216        area    181.835116    815.387885  Topic10  -5.4266   2.9011

[757 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
13        3  0.968926  academy
32        1  0.029214  account
32        2  0.391956  account
32        3  0.386400  account
32        4  0.000214  account
...     ...       ...      ...
4567      8  0.186634     year
4567      9  0.036418     year
4568      1  0.987738     yell
4575      8  0.362854     zone
4575     10  0.631505     zone

[2205 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 7, 3, 9, 1, 2, 8, 5, 4])

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def predict_topic(text, nlp=nlp, model=model):
    global sent_to_words
    global lemmatization
# Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
# Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
# Step 4: LDA Transform
    topic_probability_scores = model.transform(mytext_4)
    #topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), 1:14].values.tolist()
    #topic_guess = df_topic_keywords.iloc[np.argmax(topic_probability_scores), Topics]
    return topic_probability_scores
# Predict the topic
mytext = 'My complaint is with Carrington Mortgage Services. The company is charging property inspection payments of XXXX along with mortgage of XXXX the dates are from XX/XX/XXXX until present, also attorney fees. This makes my payments double. Carrington is also Charges a fee of XXXX unknown.'
prob_scores = predict_topic(text = mytext)

In [ ]:
def similar_documents(text, doc_topic_probs, documents = data, nlp=nlp, top_n=5, verbose=False):
    x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    x = list(x)
    print(x)
    if verbose:        
        # print("Topic KeyWords: ", topic)
        print("Topic Prob Scores of text: ", np.round(x, 1))
        print("Most Similar Doc's Probs:  ", np.round(doc_topic_probs[doc_ids], 1))
    return doc_ids, np.take(documents, doc_ids)

In [ ]:
mytext = ['I did not feel that the savings was enough to justify the fees. The several person worked numbers and got to a number that seemed reasonable. I explained that I may not be staying in the home for more than 8 months. After the refi was done I received a check that I did not ask for. I called and they stated this was mine to keep. After I started paying my mortgage I am now told that my payment is increasing over {$100.00} due to an escrow shortage. I feel that I was intentionally mislead about the payment to book the loan and that the insurance was left out to increase the loan after the loan closed. The escrow should have increased {$19.00} per month not over {$100.00} due to a change in insurance premium from {$820.00} to {$1000.00}. I was not willing to close on the loan and pay the fees unless I had substantial savings and I feel that they were deceptive just to get the extra fees. I would have never refinanced the loan for such a small savings monthly. ']
doc_ids, docs = similar_documents(text=mytext, doc_topic_probs=lda_output, documents = data, top_n=5, verbose=True)
print('\n', docs[0][:500])